In [2]:
#%pip install tensorflow

## Exercise 1

Use the following code to explore l1 and l2 regularization.  Try different combinations of l1, l2, and alpha.  What works the best?

In [3]:
#Setting up some data

import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

2024-04-16 14:16:08.181100: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 14:16:08.563290: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 14:16:08.770524: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-16 14:16:10.080899: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# You'll need to modify this to try different regularizers.  To make your life easier,
# you might want to create a function to generate your model

tf.random.set_seed(42) 

from functools import partial

RegularizedDense = partial(tf.keras.layers.Dense,
                           activation="relu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

## Exercise 2

1. Build three networks for processing the MNIST Fashion data.  Both networks should have three dense layers (but may also have dropout layers or activation function layers) with 100 neurons.

    1.  A self-normalizing neural network with dropout.
    2.  A network without normalization but using dropout.
    3.  A network with BatchNormalization.

Try each with either a 1cycle or performance schedule.  Which works the best?

In [6]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(self, iterations, max_lr=1e-3, start_lr=None,
                 last_iterations=None, last_lr=None, cutover=0.5):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.cutover = cutover
        self.half_iteration = int((iterations - self.last_iterations) * cutover)
        self.second_phase_start = self.half_iteration + ((iterations - self.last_iterations) - self.half_iteration) * cutover
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs=None):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                   self.max_lr, self.start_lr)
        else:
            lr = self._interpolate(2 * self.half_iteration, self.iterations,
                                   self.start_lr, self.last_lr)
        self.iteration += 1
        self.model.optimizer.learning_rate = lr

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="selu",
                          kernel_initializer="lecun_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="selu",
                          kernel_initializer="lecun_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

import math
batch_size=128
n_epochs=10
onecycle = OneCycleScheduler(math.ceil(len(X_train)/batch_size)*n_epochs, max_lr=0.1)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid), callbacks=lr_scheduler)

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7178 - loss: 0.7774 - val_accuracy: 0.8266 - val_loss: 0.4625 - learning_rate: 0.0100
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8042 - loss: 0.5404 - val_accuracy: 0.8450 - val_loss: 0.4285 - learning_rate: 0.0100
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8125 - loss: 0.5231 - val_accuracy: 0.8432 - val_loss: 0.4185 - learning_rate: 0.0100
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8219 - loss: 0.4933 - val_accuracy: 0.8532 - val_loss: 0.4047 - learning_rate: 0.0100
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8224 - loss: 0.4952 - val_accuracy: 0.8150 - val_loss: 0.4731 - learning_rate: 0.0100
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8258 - loss: 0.4772 - val_accuracy: 0.8532 - val_loss: 0.3860 - learning_rate: 0.0100
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8280 - l

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10


2024-04-16 14:40:37.998773: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6895 - loss: 0.8506 - val_accuracy: 0.8360 - val_loss: 0.4423
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8091 - loss: 0.5233 - val_accuracy: 0.8494 - val_loss: 0.3999
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8276 - loss: 0.4714 - val_accuracy: 0.8634 - val_loss: 0.3816
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8336 - loss: 0.4515 - val_accuracy: 0.8446 - val_loss: 0.4025
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8363 - loss: 0.4350 - val_accuracy: 0.8642 - val_loss: 0.3632
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8444 - loss: 0.4204 - val_accuracy: 0.8630 - val_loss: 0.3577
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8490 - loss: 0.4122 - val_accuracy: 0.8612 - val_loss: 0.3614
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8511 - loss: 0.4018 - val_accurac

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), callbacks=lr_scheduler)

Epoch 1/10


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6808 - loss: 0.8752 - val_accuracy: 0.8242 - val_loss: 0.4628 - learning_rate: 0.0100
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8088 - loss: 0.5240 - val_accuracy: 0.8412 - val_loss: 0.4165 - learning_rate: 0.0100
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8237 - loss: 0.4797 - val_accuracy: 0.8394 - val_loss: 0.4113 - learning_rate: 0.0100
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8341 - loss: 0.4484 - val_accuracy: 0.8674 - val_loss: 0.3751 - learning_rate: 0.0100
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8389 - loss: 0.4318 - val_accuracy: 0.8614 - val_loss: 0.3728 - learning_rate: 0.0100
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8432 - loss: 0.4237 - val_accuracy: 0.8692 - val_loss: 0.3571 - learning_rate: 0.0100
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8466 - loss: 0.4148

In [18]:
from keras.layers import BatchNormalization
model = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), callbacks=lr_scheduler)

/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7377 - loss: 0.7452 - val_accuracy: 0.8580 - val_loss: 0.3869 - learning_rate: 0.0100
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8203 - loss: 0.4942 - val_accuracy: 0.8606 - val_loss: 0.3638 - learning_rate: 0.0100
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8358 - loss: 0.4485 - val_accuracy: 0.8668 - val_loss: 0.3481 - learning_rate: 0.0100
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8435 - loss: 0.4286 - val_accuracy: 0.8728 - val_loss: 0.3421 - learning_rate: 0.0100
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8501 - loss: 0.4123 - val_accuracy: 0.8750 - val_loss: 0.3328 - learning_rate: 0.0100
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8499 - loss: 0.4091 - val_accuracy: 0.8814 - val_loss: 0.3213 - learning_rate: 0.0100
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8553 - l

## Exercise 3

Use hyperparameter tuning with the Fashion MNIST data to explore the following parameters:

- Dropout rate
- Number of layers (2-4)
- Number of neurons per layer (64 - 128)

Use Bayesian optimization and tensorboard to examine your results.


In [20]:
#%pip install keras_tuner

In [29]:
import keras_tuner as kt
#Load the data
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

#X_train = X_train.reshape((X_train.shape[0], -1)).astype('float32') 
#X_valid = X_valid.reshape((X_valid.shape[0], -1)).astype('float32')
#X_test = X_test.reshape((X_test.shape[0], -1)).astype('float32')

#Clear the session for a fresh start
tf.keras.backend.clear_session()
tf.random.set_seed(42)

def build_model(hp):
    # Set up parameters in the "hp" object, which is passed in by the keras tuner when iterating through
    # different models
    n_hidden = hp.Int("n_hidden", min_value=2, max_value=4, default=2)
    n_neurons = hp.Int("n_neurons", min_value=64, max_value=128)
    dropout_rate = hp.Float('rate', min_value=0.1, max_value=0.3)
    tf.keras.optimizers.SGD(learning_rate=0.1)
    
    # The following builds the model based on the parameters from the hp model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dropout(rate=dropout_rate))
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

from pathlib import Path 
root_logdir = Path(bayes_tuner.project_dir) / "tensorboard" 
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir) 
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=1, min_delta=0.01)
bayes_tuner = kt.BayesianOptimization(build_model, objective="val_accuracy", max_trials=5, overwrite=True, directory="my_fashion_mnist", project_name="my_rnd_search", seed=42)

# Finally, this kicks it all off
bayes_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb, tensorboard_cb])

Trial 5 Complete [00h 00m 27s]
val_accuracy: 0.46380001306533813

Best val_accuracy So Far: 0.7360000014305115
Total elapsed time: 00h 02m 22s


In [30]:
%load_ext tensorboard 
%tensorboard --logdir=my_fashion_mnist/my_rnd_search/tensorboard 
from IPython.display import display, HTML
# Display this inline here
display(HTML('<a href="http://localhost:6006/">http://localhost:6006/</a>'))